In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.io import loadmat, savemat

In [5]:
location = "C:/Users/alexc/OneDrive/Documents/Taiwan Classes/Second Semester/Brain Computer Interfaces/BCI Presentation and Project/SSVEP Projects/Wearable SSVEP BCI Database/Dataset"
subfolder = "/S001-S010/"   

s1_s10 = glob.glob(location+subfolder+"*.mat")
s1mat = loadmat(s1_s10[0])

In [6]:
subject1 = s1mat['data'].T #matlab has them in different order
print(subject1.shape)

channel_idx = [i for i in range(8)] # in order: POz, PO3, PO4, PO5, PO6, Oz, O1, O2
time_points = [i for i in range(710)] #710 data points for 2.84 seconds (250 sampling rate)
electrode_idx = [i for i in range(2)]  #0 for dry, 1 for wet
blocks = [i for i in range(10)]
targets = [i for i in range(12)]

# The order now is: [targets, blocks, electrode, data/time points, channel]... Slice from right to left:
subject1_target0_block0_electrode_dry = subject1[0][0][0][:][:]

new_subject1 = []

for target in targets:
    for block in blocks:
        for electrode in electrode_idx:
            new_subject1.append(subject1[target][block][electrode])

new_subject1 = np.array(new_subject1)
print(new_subject1.shape)
#new_subject1

(12, 10, 2, 710, 8)
(240, 710, 8)


array([[-52325.52005801, -18745.51288757, -21403.26925536, ...,
         -9890.82514589, -21657.90031301, -18416.8081532 ],
       [-53157.22841962, -18924.99738485, -22218.03499568, ...,
         -9916.03791213, -21772.43064478, -18462.49511614],
       [-55446.94098514, -18735.45460316, -23016.528667  , ...,
         -9598.10671795, -21580.20565392, -18162.22179903],
       ...,
       [-54221.93135154, -18470.40763321, -22420.31827094, ...,
         -9379.10433883, -21288.96244102, -17904.81912522],
       [-55603.44789049, -18182.33836784, -22545.44332894, ...,
         -9067.5210397 , -21014.75125639, -17616.21341802],
       [-54680.81263785, -18041.43297919, -21743.68630312, ...,
         -9066.49285951, -20945.59496317, -17577.50019893]])

In [7]:
new_subject1 = new_subject1.reshape(240*710,8)

print(new_subject1.shape)

(170400, 8)


In [8]:
data = pd.DataFrame(new_subject1)

data["Electrode_Type"] = ""
data = np.array(data)

for i in range(len(data)):
    if(i%2==0): 
        data[i,8] = "Dry"
    else:
        data[i,8] = "Wet"

In [9]:
data = pd.DataFrame(data)
data["Target"] = np.NaN
data = np.array(data)

# 14200 because 170400 (all the rows) divided by 12 targets = 14200

start = 0
actual = 14200
for i in range(len(targets)):
    data[start:actual,9] = targets[i]
    start = actual
    actual += 14200

# if i also want to creat a Blocks columns, then for blocks would be 170400/120 = 1420
data = pd.DataFrame(data)
data["Block"] = np.NaN
data = np.array(data)

start = 0
actual = 1420
for t in targets:
    for i in range(len(blocks)):
        data[start:actual,10] = blocks[i]
        start = actual
        actual += 1420

In [11]:
data = pd.DataFrame(data)
data = data.rename(columns = {8: "Electrode_Type", 9: "Target", 10:"Block"})
data

,0,1,2,3,4,5,6,7,Electrode_Type,Target,Block
0,-52325.520058,-18745.512888,-21403.269255,-15633.971431,-20347.417614,-9890.825146,-21657.900313,-18416.808153,Dry,0,0
1,-53157.22842,-18924.997385,-22218.034996,-15684.843998,-20429.269697,-9916.037912,-21772.430645,-18462.495116,Wet,0,0
2,-55446.940985,-18735.454603,-23016.528667,-15365.258775,-20164.356838,-9598.106718,-21580.205654,-18162.221799,Dry,0,0
3,-55945.653078,-18482.03054,-22753.001616,-15137.58392,-19940.481778,-9375.259839,-21344.08184,-17942.995902,Wet,0,0
4,-54100.159055,-18461.198715,-21738.858327,-15286.223011,-20024.74785,-9530.604454,-21378.1906,-18058.777932,Dry,0,0
...,...,...,...,...,...,...,...,...,...,...,...
170395,12259.662882,11984.289407,11000.365675,11004.657209,6387.859964,13515.428514,14307.30607,13128.832765,Wet,11,9
170396,12163.639794,11957.243798,10855.884007,10989.636838,6245.568767,13490.215748,14325.992127,13112.73951,Dry,11,9
170397,12169.183026,11944.637414,10865.53996,10982.797204,6247.759238,13501.033992,14342.487713,13133.794852,Wet,11,9
170398,12247.369423,11972.308872,10988.519251,10995.582401,6361.574314,13502.017468,14304.48975,13105.452842,Dry,11,9


In [27]:
#Saving into mat file (without Target, Block, and Electrode info)

data_mat = data.iloc[:,0:8].T
data_mat = np.array(data_mat)

folder_destination = "/Mat_files_alex/"   

data_dic = {'__header__':  'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Today hehe',
            '__version__': '1.0',
            '__globals__': [],
            "data": data_mat}

savemat(location+folder_destination+"Subject1.mat", data_dic)

In [29]:
data_dic['data']

array([[-52325.52005800724, -53157.22841961553, -55446.94098513967, ...,
        12169.18302590746, 12247.369423351849, 12306.556839141658],
       [-18745.51288756805, -18924.997384846054, -18735.45460316268, ...,
        11944.637414493407, 11972.308872479729, 11994.750022575261],
       [-21403.26925535615, -22218.03499567717, -23016.528667004626, ...,
        10865.539959999322, 10988.519250662283, 11076.674525539549],
       ...,
       [-9890.825145889827, -9916.037912132615, -9598.106717950506, ...,
        13501.033991636872, 13502.017468334287, 13518.781275676565],
       [-21657.90031301381, -21772.430644776265, -21580.20565391813, ...,
        14342.487713246806, 14304.489749937638, 14288.977640210249],
       [-18416.808153200636, -18462.495116144128, -18162.22179902922,
        ..., 13133.794852125338, 13105.452841845325, 13106.123394139016]],
      dtype=object)